In [1]:
from rdflib import Graph, Namespace, URIRef
from rdflib.namespace import RDF, RDFS, OWL
from owlrl import DeductiveClosure, OWLRL_Semantics
from intension import Intension
from tqdm import tqdm
import pandas as pd
import json, os

In [2]:
MODELS = [ 
    { "model_name": "gpt-3.5-turbo", "batch_size": 50 },
    { "model_name": "gpt-4o-2024-05-13", "batch_size": 50 },
    { "model_name": "gpt-4-0125-preview", "batch_size": 50 },
    { "model_name": "mistralai/Mistral-7B-Instruct-v0.3", "batch_size": 50 },
    { "model_name": "claude-3-5-sonnet-20240620", "batch_size": 1 },
    { "model_name": "mistralai/Mixtral-8x7B-Instruct-v0.1", "batch_size": 50 },
    { "model_name": "claude-3-opus-20240229", "batch_size": 1 },
    { "model_name": "meta-llama/Meta-Llama-3-70B-Instruct", "batch_size": 50 },
    { "model_name": "claude-3-haiku-20240307", "batch_size": 1 },
]

In [3]:
# Define ex: namespace
EX = Namespace("http://example.org/")

# Define URIs of individuals of interest
individuals = [
    URIRef('x1', base="http://example.org/"),
    URIRef('y1', base="http://example.org/"),
    URIRef('y2', base="http://example.org/")
]

# Define base knowledge graph in Turtle serialization
triples = '''
@prefix ex: <http://example.org/> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix owl: <http://www.w3.org/2002/07/owl#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

ex:x1 ex:r2 ex:y1 .
ex:x1 ex:r2 ex:y2 .
ex:r2 rdfs:domain ex:t1 .
ex:r2 rdfs:range ex:t2 .
ex:r2 rdf:type owl:FunctionalProperty .
'''

# Create a new graph
graph = Graph()
graph.bind("ex", EX)
graph.bind("rdf", RDF)
graph.bind("rdfs", RDFS)
graph.bind("owl", OWL)
graph.parse(data=triples, format='turtle')

# Create another graph to store deductive closure
closure = Graph()
closure.bind("ex", EX)
closure.bind("rdf", RDF)
closure.bind("rdfs", RDFS)
closure.bind("owl", OWL)
closure += graph
DeductiveClosure(OWLRL_Semantics).expand(closure)

# Create a graph to store set difference between closure and graph (i.e. the inferred triples)
inferred = Graph()
inferred.bind("ex", EX)
inferred.bind("rdf", RDF)
inferred.bind("rdfs", RDFS)
inferred.bind("owl", OWL)
inferred += (closure - graph)

# Create a graph to store inferred triples with individuals of interest as subjects or objects
interesting = Graph()
interesting.bind("ex", EX)
interesting.bind("rdf", RDF)
interesting.bind("rdfs", RDFS)
interesting.bind("owl", OWL)
for i in individuals:
    for s, p, o in inferred.triples((i, None, None)):
        interesting.add((s,p,o))
    for s, p, o in inferred.triples((None, None, i)):
        interesting.add((s,p,o))

# Print triples of interest directly
for s, p, o in interesting.triples((None, None, None)):
    print(f'<{s}>, <{p}>, <{o}>')


<http://example.org/x1>, <http://www.w3.org/1999/02/22-rdf-syntax-ns#type>, <http://example.org/t1>
<http://example.org/y1>, <http://www.w3.org/2002/07/owl#sameAs>, <http://example.org/y1>
<http://example.org/x1>, <http://www.w3.org/2002/07/owl#sameAs>, <http://example.org/x1>
<http://example.org/y2>, <http://www.w3.org/1999/02/22-rdf-syntax-ns#type>, <http://example.org/t2>
<http://example.org/y2>, <http://www.w3.org/2002/07/owl#sameAs>, <http://example.org/y2>
<http://example.org/y1>, <http://www.w3.org/1999/02/22-rdf-syntax-ns#type>, <http://example.org/t2>
<http://example.org/y2>, <http://www.w3.org/2002/07/owl#sameAs>, <http://example.org/y1>
<http://example.org/y1>, <http://www.w3.org/2002/07/owl#sameAs>, <http://example.org/y2>


In [4]:
queries = [ { "s": str(s), "p": str(p), "o": str(o), "graph": triples } for s, p, o in interesting.triples((None, None, None)) ]

In [5]:
for model in MODELS:
    filename = f'experiments/{model["model_name"].split("/")[-1]}-owl-inf.json'
    if os.path.isfile(filename):
        print(f'{model["model_name"]:36}: EXISTS')
    else:
        results = []
        batches = [ queries[i:i+model["batch_size"]] for i in range(0, len(queries), model["batch_size"]) ] 
        intension = Intension(model=model["model_name"])
        for batch in tqdm(batches, desc=f'{model["model_name"]:36}', total=len(batches)):
            response = intension.chain.batch(batch)
            for i, result in enumerate(response):
                result["model"] = model["model_name"]
                result["rationale"] = result["text"]["rationale"]
                result["predicted"] = result["text"]["answer"]
                result.pop("text")
            results.extend(response)
        json.dump(results, open(filename, "w+"))

/Users/bradleyallen/Documents/GitHub/predictable-logical-inference-with-llm-intensions/env/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(
gpt-4-0125-preview                  : 100%|██████████| 1/1 [00:16<00:00, 16.53s/it]


The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to /Users/bradleyallen/.cache/huggingface/token
Login successful


claude-3-5-sonnet-20240620          : 100%|██████████| 8/8 [00:45<00:00,  5.71s/it]


The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to /Users/bradleyallen/.cache/huggingface/token
Login successful


claude-3-opus-20240229              : 100%|██████████| 8/8 [02:10<00:00, 16.31s/it]


The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to /Users/bradleyallen/.cache/huggingface/token
Login successful


claude-3-haiku-20240307             : 100%|██████████| 8/8 [00:23<00:00,  2.97s/it]
